In [ ]:
pip install tabulate

In [ ]:
pip install psycopg2

In [ ]:
# With BigQuery DataFrames, you can use many familiar Pandas methods, but the
# processing happens BigQuery rather than the runtime, allowing you to work with larger
# DataFrames that would otherwise not fit in the runtime memory.
# Learn more here: https://cloud.google.com/python/docs/reference/bigframes/latest

import bigframes.pandas as bf

bf.options.bigquery.location = "us" #this variable is set based on the dataset you chose to query
bf.options.bigquery.project = "Meaku AI Test Project" #this variable is set based on the dataset you chose to query

<ipython-input-80-0b6cc4f77bfc>:8: UnknownLocationWarning: The location 'us' is set to an unknown value. Did you mean 'US'?
  bf.options.bigquery.location = "us" #this variable is set based on the dataset you chose to query


In [ ]:
import requests
from bs4 import BeautifulSoup

url = "https://help.hackerearth.com/assessments-recruiters"
html = requests.get(url)
#print(html.text)


In [ ]:
import requests
from bs4 import BeautifulSoup

visited_urls = set()  # Track URLs that have been visited
links_list = []  # Initialize list to store scraped links

# Function to scrape and follow links
def scrape_page(url):
    if url in visited_urls or '#' in url:  # Avoid revisiting URLs with #
        return

    print(f"Scraping: {url}")
    visited_urls.add(url)  # Add the URL to the visited set
    links_list.append(url)

    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find and scrape all internal links
        for link in soup.find_all('a', href=True):
            href = link['href']
            if href.startswith('/') or url.startswith(href):
                full_url = requests.compat.urljoin(url, href)
                if full_url not in visited_urls:  # Check to avoid revisiting
                    scrape_page(full_url)
    except Exception as e:
        print(f"Failed to scrape {url}: {e}")

# Example list of URLs to start scraping
start_urls = ["https://help.hackerearth.com/"]

# Loop over start URLs and scrape links
for start_url in start_urls:
    scrape_page(start_url)

# Display the first few links to verify
print(links_list)  # Adjust the number as needed


Scraping: https://help.hackerearth.com/
Scraping: https://help.hackerearth.com/assessments-recruiters
Scraping: https://help.hackerearth.com/facecode
Scraping: https://help.hackerearth.com/assessments-ats-integrations
Scraping: https://help.hackerearth.com/assessments-product-updates
Scraping: https://help.hackerearth.com/assessments-best-practices
Scraping: https://help.hackerearth.com/sso
Scraping: https://help.hackerearth.com/upskilling
Scraping: https://help.hackerearth.com/assessments-candidates
Scraping: https://help.hackerearth.com/problem-setting-for-hackerearth
Scraping: https://help.hackerearth.com/campus
Scraping: https://help.hackerearth.com/hiring-challenges
Scraping: https://help.hackerearth.com/frequently-asked-questions-faqs
['https://help.hackerearth.com/', 'https://help.hackerearth.com/assessments-recruiters', 'https://help.hackerearth.com/facecode', 'https://help.hackerearth.com/assessments-ats-integrations', 'https://help.hackerearth.com/assessments-product-updates'

In [ ]:
import requests
from bs4 import BeautifulSoup

# Ensure the links_list variable is accessible in this cell
# You can directly use links_list from Cell 1 if it was defined globally

article_List = []
# Print all <a> tags that match specific criteria
for link in links_list:
    try:
        response = requests.get(link)
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find <a> tags with a specific href format or containing specific text
        a_tags = soup.find_all('a', href=True)
        for a_tag in a_tags:
            href = a_tag['href']
            if href.startswith("https://help.hackerearth.com/hc/en-us/articles/"):
                article_List.append(href)

            # Optionally, add more criteria for filtering if needed
    except Exception as e:
        print(f"Failed to fetch {link}: {e}")

print(article_List);


['https://help.hackerearth.com/hc/en-us/articles/360003266273-overview', 'https://help.hackerearth.com/hc/en-us/articles/360003362373-logging-into-hackerearth-assessment', 'https://help.hackerearth.com/hc/en-us/articles/360003352433-administrator-dashboard', 'https://help.hackerearth.com/hc/en-us/articles/360003357253-test-dashboard', 'https://help.hackerearth.com/hc/en-us/articles/360003336934-setting-up-your-account', 'https://help.hackerearth.com/hc/en-us/articles/360003337074-personal-settings', 'https://help.hackerearth.com/hc/en-us/articles/360003371513-setting-your-personal-information', 'https://help.hackerearth.com/hc/en-us/articles/360003178554-changing-your-password', 'https://help.hackerearth.com/hc/en-us/articles/360004396114-ats-integrations', 'https://help.hackerearth.com/hc/en-us/articles/360003337694-company-settings', 'https://help.hackerearth.com/hc/en-us/articles/360003408193-company-information', 'https://help.hackerearth.com/hc/en-us/articles/360003388274-creating

In [ ]:
import requests
from bs4 import BeautifulSoup

# Initialize a list to store the scraped data
scraped_data = []

# Function to scrape the content of each link and store it
def scrape_content(links):
    for link in links:
        try:
            response = requests.get(link)
            soup = BeautifulSoup(response.content, 'html.parser')

            # Extract title (if available)
            title = soup.find('h1').get_text(strip=True) if soup.find('h1') else "No Title"

            # Extract content from all heading (h1-h6), paragraph (p), and list (ol, ul, li) tags
            content_tags = soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'p', 'ol', 'ul', 'li'])

            content_list = []
            for tag in content_tags:
                if tag.name == 'ol' or tag.name == 'ul':
                    # Handle ordered/unordered list items
                    items = tag.find_all('li')
                    for idx, item in enumerate(items, 1):
                        content_list.append(f"{idx}. {item.get_text(strip=True)}")
                else:
                    # Handle headings and paragraphs
                    content_list.append(tag.get_text(strip=True))

            # Join all extracted content into a single string
            content = "\n".join(content_list)

            # Add the data to the scraped_data list as a dictionary
            scraped_data.append({
                'url': link,
                'title': title,
                'content': content if content else "No Content"
            })
            print(f"Scraped content from: {link}")

        except Exception as e:
            print(f"Failed to scrape {link}: {e}")

# Call the function to scrape all links
scrape_content(article_List)

# # Display a few entries to verify
# for data in scraped_data[:5]:  # Adjust the number as needed
#     print(data)


Scraped content from: https://help.hackerearth.com/hc/en-us/articles/360003266273-overview
Scraped content from: https://help.hackerearth.com/hc/en-us/articles/360003362373-logging-into-hackerearth-assessment
Scraped content from: https://help.hackerearth.com/hc/en-us/articles/360003352433-administrator-dashboard
Scraped content from: https://help.hackerearth.com/hc/en-us/articles/360003357253-test-dashboard
Scraped content from: https://help.hackerearth.com/hc/en-us/articles/360003336934-setting-up-your-account
Scraped content from: https://help.hackerearth.com/hc/en-us/articles/360003337074-personal-settings
Scraped content from: https://help.hackerearth.com/hc/en-us/articles/360003371513-setting-your-personal-information
Scraped content from: https://help.hackerearth.com/hc/en-us/articles/360003178554-changing-your-password
Scraped content from: https://help.hackerearth.com/hc/en-us/articles/360004396114-ats-integrations
Scraped content from: https://help.hackerearth.com/hc/en-us/ar

In [ ]:
from google.cloud import bigquery
import pandas as pd

# Initialize a BigQuery client
client = bigquery.Client()

# Convert the scraped data to a pandas DataFrame
df = pd.DataFrame(scraped_data)

# Define your BigQuery table information
project_id = 'ai-chatbot-assignment'
dataset_id = 'myDataset1'
table_id = 'Table1'

# Specify the table destination
table_ref = f"{project_id}.{dataset_id}.{table_id}"

# Load the DataFrame to BigQuery
job = client.load_table_from_dataframe(df, table_ref)

# Wait for the load job to complete
job.result()

print(f"Data loaded to BigQuery table {table_ref}.")


Data loaded to BigQuery table ai-chatbot-assignment.myDataset1.Table1.


In [ ]:
import bigframes.pandas as bf
bf.close_session()

In [ ]:
print(len(article_List))

398
